In [1]:
import sys
import pandas
import numpy as np
import statsmodels.formula.api as smf
import scipy.stats as stats
from importlib import reload
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.utils import check_array
sys.path.insert(0,'/home/users/jvogel/Science/scripts/')
import kfold_learning as kfl

In [5]:
reload(kfl)

<module 'kfold_learning' from '/Users/jakevogel/git/hack_projects/kfold_learning.py'>

In [2]:
xdf = pandas.read_csv('/Users/jakevogel/Science/TADPOLE/ISVD_75_cleaned.csv')
ydf = pandas.read_csv('/Users/jakevogel/Science/TADPOLE/ISVD_75_cleaned_ref.csv')

In [3]:
tps = {}
for rid in ydf.RID.unique():
    if type(ydf[ydf.RID==rid]['RID']) == str:
        tps.update({rid: 1})
    else:
        tps.update({rid: len(ydf[ydf.RID==rid]['RID'])})

In [21]:
max(list(tps.values()))

19

In [24]:
len(np.array(list(tps.values()))[np.array(list(tps.values()))>1])

1667

In [67]:
i = 0
held_idx = [ydf[ydf.RID==x].index[i] for x in ydf.RID.unique() if tps[x]>(i+1)]

In [29]:
len(held_idz)

1667

In [39]:
ydf.columns[-5:]

Index(['DX_bl', 'y_DX', 'y_ADAS11', 'y_ADAS13', 'ventr_ICV_ratio'], dtype='object')

In [36]:
#for col in ydf.columns[-5:]:
#    xdf[col] = ydf[col]

In [70]:
test = xdf.loc[held_idx]
tdf = ydf.loc[test.index]

In [76]:
x = tdf.RID.unique()[0]
#tdf[tdf.RID==x].index[i+ii]
tdf[tdf.RID==x]


,Unnamed: 0,RID,PTID,VISCODE,SITE,D1,D2,COLPROT,ORIGPROT,EXAMDATE,DX_bl,y_DX,y_ADAS11,y_ADAS13,ventr_ICV_ratio
0,0,2,011_S_0002,bl,11,1,1,ADNI1,ADNI1,2005-09-08,CN,1.0,-0.08395,0.098839,1.771697


In [86]:
ydf.loc[ydf[ydf.RID==x].index[i+ii]]['y_ADAS13']

0.18433181140532068

In [149]:
i = 0
ii = 2
t_y = pandas.Series([ydf.loc[ydf[ydf.RID==x
                                        ].index[i+ii]]['y_ADAS13'] if tps[x]>(i+ii) else np.nan for x in tdf.RID.unique()
                               ]).dropna()
test = pandas.DataFrame(xdf.loc[held_idx].reset_index(drop=True), copy = True).loc[t_y.index]

In [151]:
#test.shape
t_y

0       0.212544
1       1.494935
2       0.383530
3      -0.357846
4       0.697288
5       2.036104
6      -1.069846
7       1.523147
8      -0.870648
9      -0.813368
10     -0.443185
11     -1.098059
12     -0.927073
13     -0.357692
14     -0.813368
15      1.608640
16     -0.813368
17      0.697288
18     -0.785155
19     -0.471397
20     -0.727875
21      0.810993
22     -0.214919
23     -0.687340
24      1.238457
25      0.839206
26     -1.012566
27      1.238457
28     -0.214919
29      1.010191
          ...   
1629    0.041559
1630    1.152964
1631   -0.727875
1632   -1.411817
1633    0.981979
1634   -0.727875
1635   -1.240831
1636   -0.642383
1637   -0.385904
1638   -1.326324
1639   -1.155339
1640   -0.813368
1641    2.264369
1642   -1.411817
1643   -0.707665
1644   -0.471397
1645   -1.155339
1646    1.323949
1647   -1.155339
1648   -0.471397
1649   -0.471397
1652   -0.129426
1654   -1.069846
1656   -0.727875
1659   -1.069846
1660   -1.240831
1661    0.298037
1662    0.6400

In [135]:
all(test.index == t_y.index)

True

In [129]:
xdf.loc[held_idx].reset_index(drop=True)

,Unnamed: 0,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,CDRSB,MMSE,...,ST97TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST98CV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST98SA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST98TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST98TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99CV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99SA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST9SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16
0,0,0.069575,-0.874880,0.004105,-0.172768,-0.253396,-0.486480,-0.819258,-0.772763,0.356549,...,1.439562,-0.547293,-0.209333,-0.836143,-0.023183,3.176360,2.627168,1.421115,-0.606512,3.287650
1,1,1.073477,-0.874880,0.710920,-0.172768,-0.253396,-0.486480,0.705575,0.828688,-1.669659,...,-0.118874,-0.763491,0.269184,-1.464361,-0.492644,0.169014,0.420666,-0.533728,-0.132373,1.716909
2,5,-0.905643,-0.874880,-2.116339,4.591811,-0.253396,-0.486480,-0.819258,-0.416885,0.103273,...,-0.164266,-0.231419,0.243549,-0.356837,-1.328287,0.605692,1.071538,-0.027816,-0.837174,-1.111001
3,10,-0.016473,-0.874880,0.004105,-0.172768,-0.253396,-0.486480,-0.819258,-0.772763,0.609824,...,0.440952,1.340932,0.841696,1.332369,-0.370584,0.845894,0.584538,0.947581,-0.465551,-0.701054
4,15,0.944404,1.143014,-1.056117,-0.172768,-0.253396,-0.486480,-0.819258,-0.594824,-0.403279,...,0.486343,-1.610034,-1.837147,-0.538322,-0.445698,-0.254604,-0.814145,0.903061,0.393025,-1.546838
5,20,0.227331,-0.874880,-2.116339,4.591811,1.366533,-0.486480,0.705575,1.362505,-1.669659,...,-0.542527,-0.717163,-0.200788,-0.575550,-0.342417,-0.579416,-0.389462,-0.614673,-0.580883,-0.826196
6,22,1.532403,1.143014,0.710920,-0.172768,-0.253396,0.780795,-0.819258,-0.772763,0.356549,...,-0.875397,-1.108847,-1.106553,0.080588,-0.680429,0.215578,0.921514,-0.307079,-0.529624,-0.017811
7,24,0.012210,1.143014,-1.409524,-0.172768,-0.253396,-0.486480,0.705575,1.006627,-0.656555,...,-0.875397,-1.497723,-1.239000,-0.380104,-2.023090,0.067936,0.540685,-0.222086,-1.093465,-1.531016
8,28,0.671916,1.143014,-1.409524,4.591811,-0.253396,2.048069,-0.819258,-0.772763,0.609824,...,0.213995,-1.671805,-1.606433,-0.519708,0.671621,0.229206,-0.202510,0.700697,0.226435,-1.171414
9,33,1.001770,-0.874880,0.710920,-0.172768,-0.253396,-0.486480,0.705575,-0.772763,0.609824,...,-0.315570,-0.340922,-0.269148,-0.505748,0.173991,-0.054720,0.538377,-0.647052,-0.196445,-0.795989


In [146]:
results = pandas.DataFrame(np.zeros((20,1)),columns = ['n'])
weight_mate = {}
for i in range(max(list(tps.values()))):
    weight_mate.update({i: {}})
    print('>'*10,'working on iteration %s of %s'%(i,max(list(tps.values()))),'<'*10)
    results.ix[i,'n'] = len(np.array(list(tps.values()))[np.array(list(tps.values()))>(i+1)])
    held_idx = [ydf[ydf.RID==x].index[i] for x in ydf.RID.unique() if tps[x]>(i+1)]
    train = xdf.drop(xdf.index[held_idx], axis=0)
    y = ydf.loc[train.index]['y_ADAS13']
    test = xdf.loc[held_idx]
    t_y = ydf.loc[test]['y_ADAS13']
    
    
    
    tdf = ydf.loc[held_idx]
    for ii in range(1,max(list(tps.values()))-(i+1)):
        print('*'*5,ii,'timepoints ahead...','*'*5)
        t_y = pandas.Series([ydf.loc[ydf[ydf.RID==x
                                        ].index[i+ii]]['y_ADAS13'] if tps[x]>(i+ii) else np.nan for x in tdf.RID.unique()
                               ]).dropna()
        test = pandas.DataFrame(xdf.loc[held_idx].reset_index(drop=True), copy = True).loc[t_y.index]
        output = kfl.kfold_feature_learning(train, test, y, t_y, scale = False, weighted = True)
        results.ix[i,'validation_%s'%(ii)] = output[0][0]
        results.ix[i,'prediction_%s'%(ii)] = output[0][1]
        weight_mate[i].update({ii: output[1]})
    

>>>>>>>>>> working on iteration 0 of 19 <<<<<<<<<<


/home/users/jvogel/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


***** 1 timepoints ahead... *****
running model for fold 1 of 10
running model for fold 2 of 10
running model for fold 3 of 10
running model for fold 4 of 10
running model for fold 5 of 10
running model for fold 6 of 10
running model for fold 7 of 10
running model for fold 8 of 10
running model for fold 9 of 10
running model for fold 10 of 10
152 features selected
validation prediction accuracy is 88.3400422841 percent 
 p = 0.0 
 r = 0.939893835941
testing prediction accuracy is 81.7253045417 percent 
 p = 0.0 
 r = 0.904020489489
***** 2 timepoints ahead... *****
running model for fold 1 of 10
running model for fold 2 of 10
running model for fold 3 of 10
running model for fold 4 of 10
running model for fold 5 of 10
running model for fold 6 of 10
running model for fold 7 of 10
running model for fold 8 of 10
running model for fold 9 of 10
running model for fold 10 of 10
152 features selected
validation prediction accuracy is 88.3400422841 percent 
 p = 0.0 
 r = 0.939893835941
testing 

KeyboardInterrupt: 

In [7]:
results = pandas.DataFrame(np.zeros((20,1)),columns = ['n'])
weight_mate = pandas.DataFrame(np.zeros((20,xdf.shape[-1])))
for i in range(max(list(tps.values()))):
    print('>'*10,'working on iteration %s of %s'%(i,max(list(tps.values()))),'<'*10)
    results.ix[i,'n'] = len(np.array(list(tps.values()))[np.array(list(tps.values()))>(i+1)])
    held_idx = [ydf[ydf.RID==x].index[i] for x in ydf.RID.unique() if tps[x]>(i+1)]
    train = xdf.drop(xdf.index[held_idx], axis=0)
    y = ydf.loc[train.index]['y_ADAS13']
    test = xdf.loc[held_idx]
    t_y = ydf.loc[test.index]['y_ADAS13']
    output = kfl.kfold_feature_learning(train, test, y, t_y, scale = False, weighted = True)
    results.ix[i,'validation'] = output[0][0]
    results.ix[i,'test_same_tp'] = output[0][1]
    weight_mate.loc[i] = output[1]
    
    tdf = ydf.loc[held_idx]
    for ii in range(1,max(list(tps.values()))-(i+1)):
        print('*'*5,ii,'timepoints ahead...','*'*5)
        t_y = pandas.Series([ydf.loc[ydf[ydf.RID==x
                                        ].index[i+ii]]['y_ADAS13'] if tps[x]>(i+ii) else np.nan for x in tdf.RID.unique()
                               ]).dropna()
        test = pandas.DataFrame(xdf.loc[held_idx].reset_index(drop=True), copy = True).loc[t_y.index]
        ntest = check_array(test,accept_sparse='csr')
        t_predicted = pandas.Series(safe_sparse_dot(ntest,np.array(output[1]).T,dense_output=True),index=test.index)
        r,p = stats.pearsonr(t_y[test.index],t_predicted)
        
        results.ix[i,'prediction_%s'%(ii)] = (r**2)*100
    

>>>>>>>>>> working on iteration 0 of 19 <<<<<<<<<<
running model for fold 1 of 10
running model for fold 2 of 10
running model for fold 3 of 10
running model for fold 4 of 10
running model for fold 5 of 10
running model for fold 6 of 10
running model for fold 7 of 10
running model for fold 8 of 10
running model for fold 9 of 10
running model for fold 10 of 10
22 features selected
validation prediction accuracy is 90.0389860449 percent 
 p = 0.0 
 r = 0.948888750302
testing prediction accuracy is 87.3361832142 percent 
 p = 0.0 
 r = 0.934538298916
***** 1 timepoints ahead... *****
***** 2 timepoints ahead... *****
***** 3 timepoints ahead... *****
***** 4 timepoints ahead... *****
***** 5 timepoints ahead... *****
***** 6 timepoints ahead... *****
***** 7 timepoints ahead... *****
***** 8 timepoints ahead... *****
***** 9 timepoints ahead... *****
***** 10 timepoints ahead... *****
***** 11 timepoints ahead... *****
***** 12 timepoints ahead... *****
***** 13 timepoints ahead... *****


ValueError: Found array with 0 sample(s) (shape=(0, 745)) while a minimum of 1 is required.

In [156]:
results

,n,validation,test_same_tp,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,prediction_8,prediction_9,prediction_10,prediction_11,prediction_12,prediction_13,prediction_14,prediction_15,prediction_16,prediction_17
0,1667.0,88.340042,87.601281,81.725305,78.675230,77.971875,77.552741,70.214987,66.503169,58.029984,50.880613,45.667146,49.006550,48.499513,47.723063,53.466512,50.809928,48.132222,40.848266,50.802742
1,1590.0,88.029616,89.554582,81.902839,80.396980,80.625941,75.847305,73.655176,67.863707,60.711688,54.020016,59.261154,59.929171,57.863543,60.080036,52.316123,55.065547,48.016969,63.752387,NaN
2,1496.0,88.274814,87.673861,83.198060,81.235273,73.938407,72.067768,64.820222,62.420851,55.477288,59.814971,58.868421,56.370232,61.406813,53.929409,59.703425,48.034166,60.989083,NaN,NaN
3,1348.0,88.157155,87.431034,82.977186,75.572777,73.172637,68.733242,66.835217,60.634825,65.908572,65.353614,61.736679,65.173333,54.724020,59.745880,50.628223,65.201271,NaN,NaN,NaN
4,1107.0,88.040709,86.422801,79.123045,77.225404,72.123232,65.152768,61.299394,67.373060,67.242661,65.836853,68.379768,57.692059,59.416867,47.610646,56.971250,NaN,NaN,NaN,NaN
5,942.0,87.907602,87.427174,81.021046,70.830861,69.465895,64.397786,70.946573,70.460121,67.862380,74.824704,64.108903,64.826620,54.975324,71.751209,NaN,NaN,NaN,NaN,NaN
6,670.0,87.946724,87.330922,78.119071,68.865914,62.450124,66.801322,70.915200,70.844873,72.677773,64.850794,69.481401,56.171503,70.241429,NaN,NaN,NaN,NaN,NaN,NaN
7,462.0,88.004570,86.450607,78.879595,72.422665,74.748042,75.446893,76.355675,76.510297,66.796467,69.005156,55.390964,71.028941,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,379.0,88.085954,86.651306,82.555468,76.841584,75.497361,73.983180,76.235010,64.501719,66.985624,57.341289,66.567022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,277.0,87.905782,89.466359,83.830951,82.271121,80.024456,82.462006,70.405961,70.304283,57.160707,72.042792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
xdf.columns[[9,25,10,8,14,219,11,13,217]]

Index(['MMSE', 'ADAS13_bl', 'RAVLT_immediate', 'CDRSB', 'FAQ',
       'ST40TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16', 'RAVLT_learning',
       'RAVLT_perc_forgetting', 'ST40CV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16'],
      dtype='object')

In [55]:
weight_mate.mean(axis=0).abs().sort_values(ascending=False)

9      0.263619
25     0.184841
10     0.146337
8      0.105768
14     0.027075
219    0.018623
11     0.002464
13     0.001271
217    0.000578
191    0.000210
23     0.000201
193    0.000174
34     0.000167
2      0.000139
12     0.000138
26     0.000110
364    0.000104
40     0.000100
344    0.000095
39     0.000086
265    0.000080
100    0.000080
224    0.000075
86     0.000074
280    0.000072
77     0.000068
3      0.000065
220    0.000063
351    0.000063
293    0.000062
         ...   
115    0.000000
140    0.000000
142    0.000000
273    0.000000
156    0.000000
167    0.000000
166    0.000000
165    0.000000
274    0.000000
275    0.000000
162    0.000000
161    0.000000
160    0.000000
159    0.000000
278    0.000000
157    0.000000
155    0.000000
284    0.000000
154    0.000000
153    0.000000
152    0.000000
282    0.000000
283    0.000000
149    0.000000
148    0.000000
147    0.000000
146    0.000000
145    0.000000
144    0.000000
113    0.000000
Length: 374, dtype: floa

In [172]:
x_years = pandas.DataFrame((xdf.values.transpose() * xdf.Years_bl.values).transpose(), columns = xdf.columns)

In [177]:
y_years = pandas.DataFrame((ydf[ydf.columns[-4:]].values.transpose() * xdf.Years_bl.values).transpose(), 
                           columns = ydf.columns[-4:])

In [181]:
y_years['RID'] = ydf['RID']

In [183]:
yrs_results = pandas.DataFrame(np.zeros((20,1)),columns = ['n'])
yrs_weight_mate = pandas.DataFrame(np.zeros((20,xdf.shape[-1])))
for i in range(max(list(tps.values()))):
    print('>'*10,'working on iteration %s of %s'%(i,max(list(tps.values()))),'<'*10)
    results.ix[i,'n'] = len(np.array(list(tps.values()))[np.array(list(tps.values()))>(i+1)])
    held_idx = [ydf[ydf.RID==x].index[i] for x in ydf.RID.unique() if tps[x]>(i+1)]
    train = x_years.drop(x_years.index[held_idx], axis=0)
    y = y_years.loc[train.index]['y_ADAS13']
    test = x_years.loc[held_idx]
    t_y = y_years.loc[test.index]['y_ADAS13']
    output = kfl.kfold_feature_learning(train, test, y, t_y, scale = False, weighted = True)
    yrs_results.ix[i,'validation'] = output[0][0]
    yrs_results.ix[i,'test_same_tp'] = output[0][1]
    yrs_weight_mate.loc[i] = output[1]
    
    tdf = y_years.loc[held_idx]
    for ii in range(1,max(list(tps.values()))-(i+1)):
        print('*'*5,ii,'timepoints ahead...','*'*5)
        t_y = pandas.Series([y_years.loc[y_years[y_years.RID==x
                                        ].index[i+ii]]['y_ADAS13'] if tps[x]>(i+ii) else np.nan for x in tdf.RID.unique()
                               ]).dropna()
        test = pandas.DataFrame(x_years.loc[held_idx].reset_index(drop=True), copy = True).loc[t_y.index]
        ntest = check_array(test,accept_sparse='csr')
        t_predicted = pandas.Series(safe_sparse_dot(ntest,np.array(output[1]).T,dense_output=True),index=test.index)
        r,p = stats.pearsonr(t_y[test.index],t_predicted)
        
        yrs_results.ix[i,'prediction_%s'%(ii)] = (r**2)*100
    

>>>>>>>>>> working on iteration 0 of 19 <<<<<<<<<<
running model for fold 1 of 10
running model for fold 2 of 10
running model for fold 3 of 10
running model for fold 4 of 10
running model for fold 5 of 10
running model for fold 6 of 10
running model for fold 7 of 10
running model for fold 8 of 10
running model for fold 9 of 10
running model for fold 10 of 10
3 features selected
validation prediction accuracy is 29.4735473146 percent 
 p = 0.0 
 r = 0.542895453238
testing prediction accuracy is 59.6735699192 percent 
 p = 0.0 
 r = 0.772486698392
***** 1 timepoints ahead... *****


/home/users/jvogel/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


***** 2 timepoints ahead... *****
***** 3 timepoints ahead... *****
***** 4 timepoints ahead... *****
***** 5 timepoints ahead... *****
***** 6 timepoints ahead... *****
***** 7 timepoints ahead... *****
***** 8 timepoints ahead... *****
***** 9 timepoints ahead... *****
***** 10 timepoints ahead... *****
***** 11 timepoints ahead... *****
***** 12 timepoints ahead... *****
***** 13 timepoints ahead... *****
***** 14 timepoints ahead... *****
***** 15 timepoints ahead... *****
***** 16 timepoints ahead... *****
***** 17 timepoints ahead... *****
>>>>>>>>>> working on iteration 1 of 19 <<<<<<<<<<
running model for fold 1 of 10
running model for fold 2 of 10
running model for fold 3 of 10
running model for fold 4 of 10
running model for fold 5 of 10
running model for fold 6 of 10
running model for fold 7 of 10
running model for fold 8 of 10
running model for fold 9 of 10
running model for fold 10 of 10
3 features selected
validation prediction accuracy is 33.1709769163 percent 
 p = 0.0 

ValueError: Found array with 0 sample(s) (shape=(0, 374)) while a minimum of 1 is required.

In [184]:
yrs_results

,n,validation,test_same_tp,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,prediction_8,prediction_9,prediction_10,prediction_11,prediction_12,prediction_13,prediction_14,prediction_15,prediction_16,prediction_17
0,0.0,29.473547,59.673570,52.431278,46.606180,30.361488,6.087687,1.439572,10.444066,9.220723,8.884591,8.834618,6.765131,6.052703,4.746801,5.173208,0.218401,16.159976,13.001290,38.703408
1,0.0,33.170977,74.315900,61.508451,40.471236,7.553595,3.055517,17.900506,22.598590,22.455931,22.056961,25.960319,30.388360,27.926805,31.523611,24.589808,38.705769,35.377511,51.060832,NaN
2,0.0,32.888376,72.389087,42.400298,6.614416,3.119255,16.657094,16.718693,9.976980,5.323247,11.561171,12.044826,9.536724,13.385344,15.931107,33.210492,24.169759,20.995667,NaN,NaN
3,0.0,31.843225,70.701964,10.874566,1.155477,9.205948,9.970846,3.644567,0.753441,1.367711,0.922844,0.621658,0.836612,0.511586,0.742548,1.044994,0.712129,NaN,NaN,NaN
4,0.0,31.270434,71.316727,4.022418,0.023178,0.059351,0.873473,2.541199,6.263413,9.825204,7.875591,5.453994,4.190468,9.077144,6.717687,2.624615,NaN,NaN,NaN,NaN
5,0.0,32.448352,85.048122,22.080120,8.096878,12.609433,13.551085,30.891100,31.166841,31.927577,40.426300,33.816600,33.410336,28.354544,7.499293,NaN,NaN,NaN,NaN,NaN
6,0.0,33.708908,87.171479,28.504324,9.785388,8.227396,17.052265,25.953804,28.075822,31.059334,27.890222,32.632710,21.705227,35.893748,NaN,NaN,NaN,NaN,NaN,NaN
7,0.0,34.486987,86.696811,46.882553,31.783368,38.606051,37.206486,41.444812,41.051973,34.939862,32.627739,27.140268,50.620408,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.0,35.387445,86.592536,67.198384,56.249239,46.996789,46.516662,46.450596,34.620023,35.232410,30.963226,37.459748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.0,34.825626,89.327814,79.375974,69.325138,64.955696,65.710269,50.201364,48.421024,38.410945,47.993691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [192]:
xdf.columns[[9,25,10,8,219,11,]]

Index(['MMSE', 'ADAS13_bl', 'RAVLT_immediate', 'CDRSB',
       'ST40TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16', 'RAVLT_learning'],
      dtype='object')

In [6]:
jnk = abs(weight_mate.mean()).sort_values(ascending=False)
xdf.columns[[x for x in range(len(jnk)) if jnk[x]>0.0001]]


Index([], dtype='object')

In [ ]:
xdf.Years_bl

In [215]:
for_stats = xdf[['CDRSB', 'MMSE', 'RAVLT_immediate', 'RAVLT_learning',
       'RAVLT_perc_forgetting', 'FAQ', 'ADAS13_bl',
       'ST40TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16', 'Years_bl']]

In [223]:
for_stats['ADAS13'] = ydf['y_ADAS13']
for_stats['RID'] = ydf['RID']

/home/users/jvogel/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/users/jvogel/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [225]:
for_stats.to_csv('ADAS13_4_mm.csv')

In [195]:
jnk = abs(yrs_weight_mate.mean()).sort_values(ascending=False)
xdf.columns[[x for x in range(len(jnk)) if jnk[x]>0.001]]


Index(['CDRSB', 'MMSE', 'RAVLT_immediate'], dtype='object')

In [199]:
grand_x = pandas.concat([xdf,x_years],axis=1)

In [200]:
grand_results = pandas.DataFrame(np.zeros((20,1)),columns = ['n'])
grand_weight_mate = pandas.DataFrame(np.zeros((20,grand_x.shape[-1])))
for i in range(max(list(tps.values()))):
    print('>'*10,'working on iteration %s of %s'%(i,max(list(tps.values()))),'<'*10)
    results.ix[i,'n'] = len(np.array(list(tps.values()))[np.array(list(tps.values()))>(i+1)])
    held_idx = [ydf[ydf.RID==x].index[i] for x in ydf.RID.unique() if tps[x]>(i+1)]
    train = grand_x.drop(grand_x.index[held_idx], axis=0)
    y = y_years.loc[train.index]['y_ADAS13']
    test = grand_x.loc[held_idx]
    t_y = y_years.loc[test.index]['y_ADAS13']
    output = kfl.kfold_feature_learning(train, test, y, t_y, scale = False, weighted = True)
    grand_results.ix[i,'validation'] = output[0][0]
    grand_results.ix[i,'test_same_tp'] = output[0][1]
    grand_weight_mate.loc[i] = output[1]
    
    tdf = y_years.loc[held_idx]
    for ii in range(1,max(list(tps.values()))-(i+1)):
        print('*'*5,ii,'timepoints ahead...','*'*5)
        t_y = pandas.Series([y_years.loc[y_years[y_years.RID==x
                                        ].index[i+ii]]['y_ADAS13'] if tps[x]>(i+ii) else np.nan for x in tdf.RID.unique()
                               ]).dropna()
        test = pandas.DataFrame(grand_x.loc[held_idx].reset_index(drop=True), copy = True).loc[t_y.index]
        ntest = check_array(test,accept_sparse='csr')
        t_predicted = pandas.Series(safe_sparse_dot(ntest,np.array(output[1]).T,dense_output=True),index=test.index)
        r,p = stats.pearsonr(t_y[test.index],t_predicted)
        
        grand_results.ix[i,'prediction_%s'%(ii)] = (r**2)*100
    

>>>>>>>>>> working on iteration 0 of 19 <<<<<<<<<<
running model for fold 1 of 10
running model for fold 2 of 10
running model for fold 3 of 10
running model for fold 4 of 10
running model for fold 5 of 10
running model for fold 6 of 10
running model for fold 7 of 10
running model for fold 8 of 10
running model for fold 9 of 10
running model for fold 10 of 10
4 features selected
validation prediction accuracy is 29.0840850466 percent 
 p = 0.0 
 r = 0.539296625676
testing prediction accuracy is 60.1258142308 percent 
 p = 0.0 
 r = 0.775408371317
***** 1 timepoints ahead... *****
***** 2 timepoints ahead... *****
***** 3 timepoints ahead... *****
***** 4 timepoints ahead... *****
***** 5 timepoints ahead... *****
***** 6 timepoints ahead... *****
***** 7 timepoints ahead... *****
***** 8 timepoints ahead... *****
***** 9 timepoints ahead... *****
***** 10 timepoints ahead... *****
***** 11 timepoints ahead... *****
***** 12 timepoints ahead... *****
***** 13 timepoints ahead... *****
*

ValueError: Found array with 0 sample(s) (shape=(0, 748)) while a minimum of 1 is required.

In [201]:
yrs_results

,n,validation,test_same_tp,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,prediction_8,prediction_9,prediction_10,prediction_11,prediction_12,prediction_13,prediction_14,prediction_15,prediction_16,prediction_17
0,0.0,29.084085,60.125814,52.787105,46.905772,30.839685,6.352402,1.467767,10.732035,9.404197,8.867628,8.576286,6.528809,5.915429,4.650797,5.065220,0.193246,15.568960,12.555583,37.801766
1,0.0,32.486970,74.973190,61.754147,41.013675,7.885218,3.167232,18.493715,23.407562,22.866540,22.217852,25.928389,30.291636,27.496764,30.845557,24.105148,37.109161,34.646920,49.539223,NaN
2,0.0,32.580476,72.499485,42.517197,6.798131,3.207663,16.857064,16.901720,9.839697,5.104636,11.124816,11.771513,9.163162,12.924037,15.620909,32.083361,23.782250,18.997328,NaN,NaN
3,0.0,31.593700,70.956686,11.034153,1.149577,9.193903,9.940574,3.608966,0.712372,1.254072,0.845764,0.543358,0.758702,0.493428,0.660138,0.984143,0.867749,NaN,NaN,NaN
4,0.0,31.078210,71.352258,4.029483,0.023080,0.059973,0.873413,2.534204,6.255359,9.823494,7.872422,5.456489,4.198331,9.065449,6.729974,2.635214,NaN,NaN,NaN,NaN
5,0.0,32.329162,85.074751,22.063627,8.084369,12.598770,13.543877,30.874127,31.155854,31.906234,40.408083,33.795287,33.382461,28.316777,7.401693,NaN,NaN,NaN,NaN,NaN
6,0.0,33.620395,87.214775,28.459186,9.766929,8.235185,17.049267,25.943285,28.031414,31.004192,27.813152,32.578157,21.631205,35.749515,NaN,NaN,NaN,NaN,NaN,NaN
7,0.0,34.384935,86.703095,46.877766,31.838138,38.635225,37.202524,41.437216,40.983406,34.878844,32.551042,27.072191,50.602972,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.0,35.269447,86.621355,67.212419,56.240047,46.971169,46.486973,46.420079,34.609265,35.233632,30.969484,37.579814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.0,34.710882,89.296862,79.326432,69.262544,64.893091,65.656089,50.166735,48.371557,38.383668,47.854490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
jnk = abs(yrs_weight_mate.mean()).sort_values(ascending=False)
xdf.columns[[x for x in range(len(jnk)) if jnk[x]>0.001]]


Index([], dtype='object')

In [206]:
yrs_weight_mate.mean().sort_values(ascending=False)

0      9.033123e-08
127    0.000000e+00
118    0.000000e+00
119    0.000000e+00
120    0.000000e+00
121    0.000000e+00
122    0.000000e+00
123    0.000000e+00
124    0.000000e+00
125    0.000000e+00
126    0.000000e+00
128    0.000000e+00
139    0.000000e+00
129    0.000000e+00
130    0.000000e+00
131    0.000000e+00
132    0.000000e+00
133    0.000000e+00
134    0.000000e+00
135    0.000000e+00
136    0.000000e+00
137    0.000000e+00
117    0.000000e+00
116    0.000000e+00
115    0.000000e+00
114    0.000000e+00
95     0.000000e+00
96     0.000000e+00
97     0.000000e+00
98     0.000000e+00
           ...     
272    0.000000e+00
273    0.000000e+00
274    0.000000e+00
275    0.000000e+00
276    0.000000e+00
277    0.000000e+00
258    0.000000e+00
256    0.000000e+00
235    0.000000e+00
255    0.000000e+00
236    0.000000e+00
237    0.000000e+00
238    0.000000e+00
239    0.000000e+00
240    0.000000e+00
241    0.000000e+00
242    0.000000e+00
243    0.000000e+00
244    0.000000e+00


In [220]:
v_results = pandas.DataFrame(np.zeros((20,1)),columns = ['n'])
v_weight_mate = pandas.DataFrame(np.zeros((20,xdf.shape[-1])))
for i in range(max(list(tps.values()))):
    print('>'*10,'working on iteration %s of %s'%(i,max(list(tps.values()))),'<'*10)
    v_results.ix[i,'n'] = len(np.array(list(tps.values()))[np.array(list(tps.values()))>(i+1)])
    held_idx = [ydf[ydf.RID==x].index[i] for x in ydf.RID.unique() if tps[x]>(i+1)]
    train = xdf.drop(xdf.index[held_idx], axis=0)
    y = ydf.loc[train.index]['ventr_ICV_ratio']
    test = xdf.loc[held_idx]
    t_y = ydf.loc[test.index]['ventr_ICV_ratio']
    output = kfl.kfold_feature_learning(train, test, y, t_y, scale = False, weighted = True)
    v_results.ix[i,'validation'] = output[0][0]
    v_results.ix[i,'test_same_tp'] = output[0][1]
    v_weight_mate.loc[i] = output[1]
    
    tdf = ydf.loc[held_idx]
    for ii in range(1,max(list(tps.values()))-(i+1)):
        print('*'*5,ii,'timepoints ahead...','*'*5)
        t_y = pandas.Series([ydf.loc[ydf[ydf.RID==x
                                        ].index[i+ii]]['ventr_ICV_ratio'] if tps[x]>(i+ii) else np.nan for x in tdf.RID.unique()
                               ]).dropna()
        test = pandas.DataFrame(xdf.loc[held_idx].reset_index(drop=True), copy = True).loc[t_y.index]
        ntest = check_array(test,accept_sparse='csr')
        t_predicted = pandas.Series(safe_sparse_dot(ntest,np.array(output[1]).T,dense_output=True),index=test.index)
        r,p = stats.pearsonr(t_y[test.index],t_predicted)
        
        v_results.ix[i,'prediction_%s'%(ii)] = (r**2)*100
    

>>>>>>>>>> working on iteration 0 of 19 <<<<<<<<<<


/home/users/jvogel/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


running model for fold 1 of 10
running model for fold 2 of 10
running model for fold 3 of 10
running model for fold 4 of 10


/home/users/jvogel/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


running model for fold 5 of 10
running model for fold 6 of 10


/home/users/jvogel/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


running model for fold 7 of 10
running model for fold 8 of 10
running model for fold 9 of 10
running model for fold 10 of 10
209 features selected
validation prediction accuracy is 84.7322028088 percent 
 p = 0.0 
 r = 0.920500965827
testing prediction accuracy is 89.9678084964 percent 
 p = 0.0 
 r = 0.948513618755
***** 1 timepoints ahead... *****
***** 2 timepoints ahead... *****
***** 3 timepoints ahead... *****
***** 4 timepoints ahead... *****
***** 5 timepoints ahead... *****
***** 6 timepoints ahead... *****
***** 7 timepoints ahead... *****
***** 8 timepoints ahead... *****
***** 9 timepoints ahead... *****
***** 10 timepoints ahead... *****
***** 11 timepoints ahead... *****
***** 12 timepoints ahead... *****
***** 13 timepoints ahead... *****
***** 14 timepoints ahead... *****
***** 15 timepoints ahead... *****
***** 16 timepoints ahead... *****
***** 17 timepoints ahead... *****
>>>>>>>>>> working on iteration 1 of 19 <<<<<<<<<<
running model for fold 1 of 10
running model 

ValueError: Found array with 0 sample(s) (shape=(0, 374)) while a minimum of 1 is required.

In [221]:
jnk = abs(v_weight_mate.mean()).sort_values(ascending=False)
xdf.columns[[x for x in range(len(jnk)) if jnk[x]>0.001]]


Index(['AGE', 'PTRACCAT', 'Hippocampus', 'WholeBrain', 'ICV', 'Ventricles_bl',
       'ICV_bl', 'VENTQC_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST10CV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST120SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST127SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST130TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST21SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST29SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST35TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST37SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST54TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST5SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST61SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST6SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST80SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
       'ST96SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16'],
      dtype='object')

In [222]:
v_results

,n,validation,test_same_tp,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,prediction_8,prediction_9,prediction_10,prediction_11,prediction_12,prediction_13,prediction_14,prediction_15,prediction_16,prediction_17
0,1667.0,84.732203,89.967808,79.032231,75.824989,86.749546,86.410480,84.381785,82.995539,82.710855,82.116661,79.566893,79.001960,75.470052,72.956446,71.815451,63.452055,58.376495,52.611451,50.187952
1,1590.0,85.827668,79.055746,75.187125,87.012708,86.055098,84.573320,82.847612,82.811810,82.702731,79.863311,80.532428,76.586525,73.536602,72.048598,63.077445,57.960202,52.096603,46.603264,NaN
2,1496.0,86.087894,75.540922,86.794993,86.035573,84.323708,83.236321,83.692743,84.309614,81.657564,82.863660,79.118219,76.054586,74.394349,64.273718,58.779631,51.600227,46.434990,NaN,NaN
3,1348.0,83.466009,88.519493,86.093270,84.722186,83.302546,83.548389,83.436428,80.537744,82.100790,78.127414,74.883356,73.756642,63.811512,59.388080,52.586250,47.888629,NaN,NaN,NaN
4,1107.0,83.372394,87.549655,84.882530,82.490186,83.154217,82.531532,80.617427,82.140784,78.090091,74.416493,74.020275,64.828871,58.901782,51.625902,47.241204,NaN,NaN,NaN,NaN
5,942.0,83.134850,87.214991,82.779821,82.263924,80.802973,79.213867,81.259914,77.246669,74.422998,73.315122,66.140467,63.691870,58.146582,52.700115,NaN,NaN,NaN,NaN,NaN
6,670.0,83.470873,85.899780,83.656243,82.184472,80.359251,81.288775,77.715298,76.461682,73.923388,66.609485,62.922655,56.033874,52.513425,NaN,NaN,NaN,NaN,NaN,NaN
7,462.0,83.824686,88.336815,83.684418,81.321355,81.693008,79.264093,77.250985,75.017662,68.445990,64.885565,57.413983,55.566672,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,379.0,84.051645,88.939382,84.484771,85.784269,80.912044,77.911807,75.017808,66.732930,62.477175,57.353163,49.869485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,277.0,84.020094,90.526421,87.474459,83.686040,80.078967,78.518662,70.056989,64.165014,57.896520,53.704165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
